Encuesta realizada durante el año 2024 a participantes de actividades de Educación Permanente en la Universidad de la República durante el año 2023.

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import pandas as pd
import numpy as np
import re

In [72]:
# Cargar el dataset
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

df = pd.read_csv('encuesta-de-participantes_2024_portal_de_datos.csv', encoding='latin1')

# Ver primeras filas
df.head()

,Id,1. Para empezar ¿Cuántos cursos de educación permanente realizó el año pasado 2023?,2. ¿Desde qué centro/facultad realizó el/los cursos? (puede marcar todas las que correspondan en caso de haber realizado dos cursos o más en diferentes centros),Centro/Facultad II,Centro/Facultad III,Centro/Facultad IV,2a. Aclaración Otro,"3. ¿Qué tipo de curso fue? (si fueron varios cursos, tome en cuenta el último curso que realizó)",4. ¿Cuánto horas/tiempo duró el curso?,5. ¿Que estrategias pedagógicas utilizó el docente para dictarlo? [Herramientas de video-conferencia (Zoom por ejemplo)],...,19. ¿Cuantos años tiene cumplidos a la fecha? (Edad en número),"20. ¿En qué departamento vive? (en caso de alumno extranjero marcar ""Otro país"" y especificar el país)",20a. Aclaración otro país,21. ¿En qué localidad o barrio reside?,22. ¿Cuántas horas a la semana trabaja? (Horas en números),"23. ¿En qué tipo de institución trabaja? (si trabaja en más de un lugar, marque el que considere su principal fuente de ingreso)",26. ¿Cuál es su máximo nivel de estudios alcanzado?,27. ¿En que año ingreso en la Universidad y/o centro de estudios terciarios? (año en números),"28. Y ¿En que año egreso? (año en números, si no egreso poner 0)","29. Para cerrar, ¿Sus padres (madre y/o padre) tienen estudios universitarios y/o estudios terciarios en otra institución (IPA, Magisterio, UTU terciario, etc.)?"
0,1,Dos cursos,FIC,NaN,NaN,NaN,NaN,Virtual,Entre 15-30 hs,Si,...,33.0,Montevideo,NaN,Parque Batlle,40,Privada,12. Universidad grado completo,2018,2022,"Si, tengo al menos un padre/madre universitari..."
1,2,Dos cursos,Música,NaN,NaN,NaN,NaN,Semi presencial/Bi-modal (con instancias virtu...,No lo recuerdo,Si,...,24.0,Canelones,NaN,Atlántida,0,No estoy trabajando actualmente,11. Universidad grado incompleto,2021 o 2022,Aún no,"Si, tengo al menos un padre/madre universitari..."
2,3,Dos cursos,Bellas Artes,Psicología,NaN,NaN,NaN,Presencial,Entre 15-30 hs,Si,...,35.0,Montevideo,NaN,Pocitos,25,Cooperativa,12. Universidad grado completo,2008,2015,"Si, tengo al menos un padre/madre universitari..."
3,4,Ninguno (Terminar encuesta y pasar a la pregun...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45.0,Montevideo,NaN,Malvin/ Montevideo,40,Privada,17. Maestría incompleto,2014,2020,"No, no tengo padres universitarios. Soy primer..."
4,5,Dos cursos,Sociales,NaN,NaN,NaN,NaN,Virtual,Entre 30-45 hs,Si,...,25.0,Montevideo,NaN,La Blanqueada,49,ONG / OSC,11. Universidad grado incompleto,2018,0,"No, no tengo padres universitarios. Soy primer..."


## 1) Inspección inicial

In [73]:
print("Filas, Columnas:", df.shape)
#display(df.head(3))
#display(df.tail(3))
#list(df.columns)

Filas, Columnas: (739, 56)


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 56 columns):
 #   Column                                                                                                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                                 --------------  -----  
 0   Id                                                                                                                                                                                                                                                     739 non-null    int64  
 1   1. Para empezar ¿Cuántos cursos de educación permanente realiz

In [75]:
#Cuáles son las columnas con más datos faltantes y en qué proporción
na_pct = df.isna().mean().sort_values(ascending=False)
na_pct.head(10)

,0
Centro/Facultad IV,0.998647
Medio IV,0.989175
Centro/Facultad III,0.983762
20a. Aclaración otro país,0.976996
2a. Aclaración Otro,0.968877
Medio III,0.947226
Centro/Facultad II,0.905277
"17a. Aclaración ""Otra""",0.871448
Medio II,0.772666
27. ¿En que año ingreso en la Universidad y/o centro de estudios terciarios? (año en números),0.069012


## 2) Limpieza básica

In [76]:
empty_cols = [c for c in df.columns if df[c].isna().all()]
print("Columnas completamente vacias:", len(empty_cols))
#df = df.drop(columns=empty_cols)
#df.shape

Columnas completamente vacias: 0


## 3) Transformaciones útiles


In [77]:
# Binarias Si/No en varios bloques
bin_like = [c for c in df.columns if c.startswith(('5.','6.'))]
for c in bin_like:
    df[c] = df[c].map({"Si": 1, "No": 0}).astype('float')
print("Columnas binarias mapeadas:", len(bin_like))
df[bin_like].head(3)

Columnas binarias mapeadas: 13


,5. ¿Que estrategias pedagógicas utilizó el docente para dictarlo? [Herramientas de video-conferencia (Zoom por ejemplo)],"5. ¿Que estrategias pedagógicas utilizó el docente para dictarlo? [Presentaciones (ppt, pdf, etc.)]","5. ¿Que estrategias pedagógicas utilizó el docente para dictarlo? [Herramientas interactivas (encuestas en vivo, foros, etc.)]",5. ¿Que estrategias pedagógicas utilizó el docente para dictarlo? [Trabajo colaborativo],5. ¿Que estrategias pedagógicas utilizó el docente para dictarlo? [Formularios de auto-evaluación],5. ¿Que estrategias pedagógicas utilizó el docente para dictarlo? [Evaluación entre pares (corrección de trabajos entre los participantes)],"5. ¿Que estrategias pedagógicas utilizó el docente para dictarlo? [Evaluación automática (por ejemplo, prueba/parcial por formulario en EVA)]",5. ¿Que estrategias pedagógicas utilizó el docente para dictarlo? [Rúbrica docente (Tabla que indica los criterios que se evalúan y los niveles de calidad alcanzados)],"6. Más allá de los items anteriores, en el curso existieron actividades interactivas/participativas del siguiente tipo: [Talleres]","6. Más allá de los items anteriores, en el curso existieron actividades interactivas/participativas del siguiente tipo: [Estudios de casos ]","6. Más allá de los items anteriores, en el curso existieron actividades interactivas/participativas del siguiente tipo: [Mesa redonda (por ejemplo charlas con invitados)]","6. Más allá de los items anteriores, en el curso existieron actividades interactivas/participativas del siguiente tipo: [Ateneos]","6. Más allá de los items anteriores, en el curso existieron actividades interactivas/participativas del siguiente tipo: [Otra]"
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [78]:
# Quiero saber cuántas personas completaron la encuesta
#1) Identifico la columna de la pregunta 1
col_p1 = [c for c in df.columns if c.startswith('1.')][0]

# 2) Defino el texto objetivo
target = "Ninguno (Terminar encuesta y pasar a la pregunta 18)"

# 3) Creo la máscara de coincidencia robusta (ignora mayúsculas y espacios extra)
mask = (
    df[col_p1]
      .astype(str)                # por si hay NaN
      .str.strip()                # quita espacios iniciales/finales
      .str.casefold()             # compara sin sensibilidad a mayúsculas/minúsculas
      .eq(target.strip().casefold())
)

# 4) Conteo y porcentaje
cantidad = mask.sum()
porcentaje = (cantidad / len(df)) * 100 if len(df) else 0

print("Columna P1:", col_p1)
print("Cantidad que marcaron 'Ninguno (Terminar encuesta y pasar a la pregunta 18)':", cantidad)
print(f"Porcentaje sobre el total: {porcentaje:.1f}%")

Columna P1: 1. Para empezar ¿Cuántos cursos de educación permanente realizó el año pasado 2023?
Cantidad que marcaron 'Ninguno (Terminar encuesta y pasar a la pregunta 18)': 42
Porcentaje sobre el total: 5.7%


In [79]:
df[col_p1].value_counts(dropna=False)

,count
1. Para empezar ¿Cuántos cursos de educación permanente realizó el año pasado 2023?,
Un curso,393
Dos cursos,168
Cuatro cursos o más,65
Tres cursos,65
Ninguno (Terminar encuesta y pasar a la pregunta 18),42
99,6


## 4) Análisis Estadístico

In [80]:
## Cálculos Estadísticos Descriptivos
#Usamos comandos de Pandas simples para obtener resúmenes rápidos de los datos.

# Cálculos Estadísticos Descriptivos

print("--- Resumen Estadístico de la Edad ---")
# describe() es una de las funciones más potentes para ver la distribución central
print(df['19. ¿Cuantos años tiene cumplidos a la fecha? (Edad en número)'].describe())

--- Resumen Estadístico de la Edad ---
count    738.000000
mean      39.345528
std       11.611467
min       18.000000
25%       30.000000
50%       38.000000
75%       48.000000
max       99.000000
Name: 19. ¿Cuantos años tiene cumplidos a la fecha? (Edad en número), dtype: float64


In [ ]:
print("\n--- Conteo por Tipo de Curso (Variable Categórica) ---")
# value_counts() muestra la frecuencia de cada categoría
print(df['3. ¿Qué tipo de curso fue? (si fueron varios cursos, tome en cuenta el último curso que realizó)'].value_counts(dropna=False))

## 5) Análisis Estadístico y Correlación

Vamos a analizar si existe correlación entre la duración del curso y las horas que trabaja por semana

In [81]:
df['4. ¿Cuánto horas/tiempo duró el curso?'].value_counts(dropna=False)

,count
4. ¿Cuánto horas/tiempo duró el curso?,
Entre 15-30 hs,228
No lo recuerdo,156
Entre 30-45 hs,111
15 hs o menos,110
Más de 45 hs,85
NaN,42
99,7


In [82]:
df['22. ¿Cuántas horas a la semana trabaja? (Horas en números)'].value_counts(dropna=False)

,count
22. ¿Cuántas horas a la semana trabaja? (Horas en números),
40,168
30,97
0,56
48,36
50,32
...,...
5,1
39,1
18,1


In [83]:
#LIMPIEZA, RENOMBRAMIENTO Y CONVERSIÓN A NUMÉRICA (FINAL)

# 1. Renombrar columnas clave:
df = df.rename(columns={
    '4. ¿Cuánto horas/tiempo duró el curso?': 'Duracion_Curso_cat',
    '22. ¿Cuántas horas a la semana trabaja? (Horas en números)': 'Horas_Trabajadas_Semana_num',
    # Añade más si las estás usando en el Notebook:
    '9. ¿Cuál es su edad?': 'Edad',
})

# 2. Mapeo para Duración del Curso (Numérica)
mapeo_duracion = {
    '15 hs o menos': 7.5,
    'Entre 15-30 hs': 22.5,
    'Entre 30-45 hs': 37.5,
    'Más de 45 hs': 60,
    'No lo recuerdo': np.nan     # Convertimos explícitamente a NaN
}
df['Duracion_Curso_Numerica'] = df['Duracion_Curso_cat'].map(mapeo_duracion)

# 3. Asegurar tipo de dato para el cálculo
df['Duracion_Curso_Numerica'] = df['Duracion_Curso_Numerica'].astype(float)
# Usamos .astype(float) para asegurar que la columna de horas esté lista para cálculos
df['Horas_Trabajadas_Semana_num'] = df['Horas_Trabajadas_Semana_num'].astype(float)

ValueError: could not convert string to float: '8-9 h'

In [84]:
# 1. Definir los mapeos: ¿Qué valor de texto se convierte en qué número?
mapeo_horas = {
    '8-9 h': 8.0,
    'Muchas': 40.0,

}

# 2. Aplicar la sustitución de valores
# Se utiliza el parámetro 'regex=False' para asegurar que el mapeo sea exacto.
df['Horas_Trabajadas_Semana_num'] = (
    df['Horas_Trabajadas_Semana_num']
    .replace(mapeo_horas)
)

# 3. Conversión final a float
# Si la columna contiene ahora solo números y NaNs, este paso debería funcionar.
try:
    df['Horas_Trabajadas_Semana_num'] = df['Horas_Trabajadas_Semana_num'].astype(float)
    print("✅ La columna ha sido actualizada y convertida a tipo float.")
except ValueError as e:
    print(f"🛑 Error de conversión: {e}")
    print("Aún quedan valores de texto no mapeados o caracteres extra (ej. ' h', ' -') que impiden la conversión. Debes mapearlos o eliminarlos.")

✅ La columna ha sido actualizada y convertida a tipo float.


In [85]:
df['Duracion_Curso_Numerica'] = df['Duracion_Curso_cat'].map(mapeo_duracion)

# 3. Asegurar tipo de dato para el cálculo
df['Duracion_Curso_Numerica'] = df['Duracion_Curso_Numerica'].astype(float)
# Usamos .astype(float) para asegurar que la columna de horas esté lista para cálculos
df['Horas_Trabajadas_Semana_num'] = df['Horas_Trabajadas_Semana_num'].astype(float)

In [86]:
# CÁLCULO DE CORRELACIÓN CON IMPUTACIÓN DE LA MEDIA

# 1. Definir las columnas a usar
duracion = df['Duracion_Curso_Numerica']
horas_trabajadas = df['Horas_Trabajadas_Semana_num']

# 2. GESTIÓN CRÍTICA DE NaN (IMPUTACIÓN POR LA MEDIA)
# Rellenamos los NaN con el promedio de cada columna para permitir el cálculo.
duracion_limpia = duracion.fillna(duracion.mean())
horas_trabajadas_limpia = horas_trabajadas.fillna(horas_trabajadas.mean())

# 3. Calculamos la correlación.
correlacion = duracion_limpia.corr(horas_trabajadas_limpia)

print("--- Coeficiente de Correlación de Pearson ---")
print(f"Correlación (Duración del Curso vs. Horas Trabajadas Semanales): {correlacion:.3f}")

# 4. Interpretación del resultado
if abs(correlacion) < 0.2:
    fuerza = "débil o nula"
    accion = "No hay una relación lineal significativa."
elif abs(correlacion) < 0.5:
    fuerza = "moderada"
    accion = "Existe una leve tendencia."
else:
    fuerza = "fuerte"
    accion = "Hay una clara relación lineal."

tendencia = "inversa" if correlacion < 0 else "directa"

print(f"Resultado: La correlación es {fuerza} y de tendencia {tendencia}. {accion}")

--- Coeficiente de Correlación de Pearson ---
Correlación (Duración del Curso vs. Horas Trabajadas Semanales): 0.009
Resultado: La correlación es débil o nula y de tendencia directa. No hay una relación lineal significativa.
